# Portfolio Risk Dashboard

The purpose of this project is to practice creating a dashboard to easily observe and take note of portfolio risk.
There are missing values in the dates and the 3 month Tbill is only reflecting a monthly number.

### Importing Packages

In [561]:
import numpy as np
import pandas as pd
import requests
from statistics import stdev

### Importing Data

For this project, we will be using the financial information publicly available from Voss Capital's 2021 13F. The data is publicly available at this address: https://whalewisdom.com/filer/voss-capital-llc#tabholdings_tab_link
The data available on a 13F is "[b]asically anything that is traded on an exchange as well as certain equity options and warrants, shares of closed-end investment companies and certain convertible debt securities." Source: whalewisdom.com

### About the Data in the 13F

The data tells us only what quarter the shares were purchased in and the predicted price that the fund paid for the amount. 

### About Stock Vantage API

Can provide stock data

import a mock csv of what we are looking for.

In [7]:
df = pd.read_csv("data/portfolio_csv_input.csv")

Figuring out how to index through these. 

In [9]:
df.head()

,ticker,shares,buy_price,buy_date
0,TSLA,2,4.98,4-3-21
1,DIS,2,2342.00,5-7-21
2,KO,4,324.00,2-2-21
3,BRK.B,2,1112.00,8-9-21


Let's take a look at the data types

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ticker     4 non-null      object 
 1   shares     4 non-null      int64  
 2   buy_price  4 non-null      float64
 3   buy_date   4 non-null      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 256.0+ bytes


lets practice indexing a dataframe

In [20]:
for i in df.ticker:
    print(i)

TSLA
DIS
KO
BRK.B


In [22]:
df.head()

,ticker,shares,buy_price,buy_date
0,TSLA,2,4.98,4-3-21
1,DIS,2,2342.00,5-7-21
2,KO,4,324.00,2-2-21
3,BRK.B,2,1112.00,8-9-21


### Now let's see if we can get some historical data using the Alpha Vantage API.

In [328]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=TSLA&outputsize=full&apikey=0HG27IzWW0AWYH0X'
r = requests.get(url)
data = r.json()
#print(data)

The type of this data is a dictionary.

In [329]:
type(data)

dict

so, how can we iterate through this dictionary to get what we need? We need to find all of the attributes with 'close' as the key.

In [330]:
#Exploring the methods and attributes

dir(data)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In the previous code, one of the attributes is '_iter_' which is is a method that is called when an iterator is required for a container, and it should return a new iterator object that can iterate through all the objects in the container. 

For mappings (like dictionaries), .__iter__() should iterate over the keys. This means that if you put a dictionary directly into a for loop, Python will automatically call .__iter__() on that dictionary, and you’ll get an iterator over its keys.

Source: https://realpython.com/iterate-through-dictionary-python/

In [331]:
#data

### Now we can get the dates from the dictionary.

In [332]:
data = data['Time Series (Daily)']

In [333]:
dates = data.keys()

In [334]:
dates

dict_keys(['2022-04-20', '2022-04-19', '2022-04-18', '2022-04-14', '2022-04-13', '2022-04-12', '2022-04-11', '2022-04-08', '2022-04-07', '2022-04-06', '2022-04-05', '2022-04-04', '2022-04-01', '2022-03-31', '2022-03-30', '2022-03-29', '2022-03-28', '2022-03-25', '2022-03-24', '2022-03-23', '2022-03-22', '2022-03-21', '2022-03-18', '2022-03-17', '2022-03-16', '2022-03-15', '2022-03-14', '2022-03-11', '2022-03-10', '2022-03-09', '2022-03-08', '2022-03-07', '2022-03-04', '2022-03-03', '2022-03-02', '2022-03-01', '2022-02-28', '2022-02-25', '2022-02-24', '2022-02-23', '2022-02-22', '2022-02-18', '2022-02-17', '2022-02-16', '2022-02-15', '2022-02-14', '2022-02-11', '2022-02-10', '2022-02-09', '2022-02-08', '2022-02-07', '2022-02-04', '2022-02-03', '2022-02-02', '2022-02-01', '2022-01-31', '2022-01-28', '2022-01-27', '2022-01-26', '2022-01-25', '2022-01-24', '2022-01-21', '2022-01-20', '2022-01-19', '2022-01-18', '2022-01-14', '2022-01-13', '2022-01-12', '2022-01-11', '2022-01-10', '2022-01-

In [335]:
data

{'2022-04-20': {'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 '2022-04-19': {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 '2022-04-18': {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 '2022-04-14': {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 '2022-04-13': {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 '2022-04-12': {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 '2022-04-11': {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '197857

### Now we have to figure out how to pull out the closing stock prices from the data

In [336]:
data

{'2022-04-20': {'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 '2022-04-19': {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 '2022-04-18': {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 '2022-04-14': {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 '2022-04-13': {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 '2022-04-12': {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 '2022-04-11': {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '197857

In [337]:
for key, value in data.items():
    print(key , value)


2022-04-20 {'1. open': '1030.0000', '2. high': '1034.0000', '3. low': '975.2501', '4. close': '977.2000', '5. volume': '21616141'}
2022-04-19 {'1. open': '1005.0600', '2. high': '1034.9400', '3. low': '995.3250', '4. close': '1028.1500', '5. volume': '16615944'}
2022-04-18 {'1. open': '989.0300', '2. high': '1014.9200', '3. low': '973.4100', '4. close': '1004.2900', '5. volume': '17238407'}
2022-04-14 {'1. open': '999.2900', '2. high': '1012.7099', '3. low': '982.1900', '4. close': '985.0000', '5. volume': '19474135'}
2022-04-13 {'1. open': '981.0750', '2. high': '1026.2400', '3. low': '973.0970', '4. close': '1022.3700', '5. volume': '18373737'}
2022-04-12 {'1. open': '997.6400', '2. high': '1021.1921', '3. low': '976.6001', '4. close': '986.9500', '5. volume': '21992032'}
2022-04-11 {'1. open': '980.4000', '2. high': '1008.4681', '3. low': '974.6400', '4. close': '975.9300', '5. volume': '19785735'}
2022-04-08 {'1. open': '1043.2100', '2. high': '1048.4400', '3. low': '1022.4350', '4

2018-02-26 {'1. open': '353.5000', '2. high': '359.0000', '3. low': '352.3600', '4. close': '357.4200', '5. volume': '4312871'}
2018-02-23 {'1. open': '347.8300', '2. high': '354.9900', '3. low': '347.1000', '4. close': '352.0500', '5. volume': '5790795'}
2018-02-22 {'1. open': '335.5300', '2. high': '347.4400', '3. low': '334.7500', '4. close': '346.1700', '5. volume': '6940349'}
2018-02-21 {'1. open': '336.0300', '2. high': '339.6900', '3. low': '333.1700', '4. close': '333.3000', '5. volume': '3181755'}
2018-02-20 {'1. open': '334.4700', '2. high': '340.8400', '3. low': '331.5000', '4. close': '334.7700', '5. volume': '3996951'}
2018-02-16 {'1. open': '332.5000', '2. high': '343.1200', '3. low': '331.6400', '4. close': '335.4900', '5. volume': '5585810'}
2018-02-15 {'1. open': '324.5000', '2. high': '334.1200', '3. low': '322.4000', '4. close': '334.0700', '5. volume': '5892048'}
2018-02-14 {'1. open': '320.8400', '2. high': '326.1700', '3. low': '318.5200', '4. close': '322.3100', 

2015-03-06 {'1. open': '199.2100', '2. high': '200.7500', '3. low': '192.1510', '4. close': '193.8800', '5. volume': '6712438'}
2015-03-05 {'1. open': '202.8500', '2. high': '206.1900', '3. low': '200.1500', '4. close': '200.6300', '5. volume': '4877015'}
2015-03-04 {'1. open': '199.2500', '2. high': '202.5200', '3. low': '197.2100', '4. close': '202.4350', '5. volume': '4221962'}
2015-03-03 {'1. open': '196.8100', '2. high': '200.2435', '3. low': '195.3200', '4. close': '199.5600', '5. volume': '4432329'}
2015-03-02 {'1. open': '202.7000', '2. high': '203.3400', '3. low': '195.8250', '4. close': '197.3250', '5. volume': '7922065'}
2015-02-27 {'1. open': '206.9000', '2. high': '208.5500', '3. low': '202.8000', '4. close': '203.3400', '5. volume': '3882084'}
2015-02-26 {'1. open': '204.0000', '2. high': '211.0900', '3. low': '202.2200', '4. close': '207.1900', '5. volume': '6472855'}
2015-02-25 {'1. open': '204.9400', '2. high': '207.1400', '3. low': '202.5800', '4. close': '203.7600', 

2011-05-17 {'1. open': '27.0000', '2. high': '27.0000', '3. low': '25.7200', '4. close': '25.9600', '5. volume': '1234200'}
2011-05-16 {'1. open': '27.9900', '2. high': '27.9900', '3. low': '26.5500', '4. close': '26.6000', '5. volume': '755700'}
2011-05-13 {'1. open': '28.0000', '2. high': '28.1891', '3. low': '27.3000', '4. close': '27.5500', '5. volume': '661500'}
2011-05-12 {'1. open': '27.0700', '2. high': '27.7400', '3. low': '26.6500', '4. close': '27.6700', '5. volume': '628000'}
2011-05-11 {'1. open': '28.2000', '2. high': '28.2985', '3. low': '26.9200', '4. close': '27.0700', '5. volume': '962500'}
2011-05-10 {'1. open': '28.2400', '2. high': '28.9500', '3. low': '27.9100', '4. close': '28.3300', '5. volume': '1535300'}
2011-05-09 {'1. open': '27.0000', '2. high': '28.0000', '3. low': '26.8500', '4. close': '27.9100', '5. volume': '916400'}
2011-05-06 {'1. open': '26.9000', '2. high': '27.7000', '3. low': '26.6200', '4. close': '27.1200', '5. volume': '981700'}
2011-05-05 {'1

In [338]:
prices = []
for value in data.values():
    prices.append(value)

In [339]:
prices

[{'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '19785735'},
 {'1. open': '1043.2100',
  '2. high': '1048.4400',
  '3. low': '1022.4350',
  '4. close': '

### Now that we have isolated the prices, we need to put them into dataframes and join them.

It is a list and we need to turn it into a dataframe.

In [340]:
type(prices)

list

In [341]:
stock_df = pd.DataFrame(prices)
stock_df.head()

,1. open,2. high,3. low,4. close,5. volume
0,1030.0000,1034.0000,975.2501,977.2000,21616141
1,1005.0600,1034.9400,995.3250,1028.1500,16615944
2,989.0300,1014.9200,973.4100,1004.2900,17238407
3,999.2900,1012.7099,982.1900,985.0000,19474135
4,981.0750,1026.2400,973.0970,1022.3700,18373737


In [342]:
stock_df = stock_df.rename(columns = {'4. close':'close'})

In [343]:
dates_df = pd.DataFrame(dates)
dates_df.head()

,0
0,2022-04-20
1,2022-04-19
2,2022-04-18
3,2022-04-14
4,2022-04-13


### Now the DataFrames can be joined. 

In [368]:
whole_df = dates_df.join(stock_df)
whole_df = whole_df[[0,'close']]
whole_df = whole_df.rename(columns = {0:'date'})

In [369]:
whole_df.head()

,date,close
0,2022-04-20,977.2000
1,2022-04-19,1028.1500
2,2022-04-18,1004.2900
3,2022-04-14,985.0000
4,2022-04-13,1022.3700


### The new DataFrame can be checked with the original dictionary to confirm that the DataFrames were properly joined and have the correct data.

Yep! Looking good.

In [370]:
data

{'2022-04-20': {'1. open': '1030.0000',
  '2. high': '1034.0000',
  '3. low': '975.2501',
  '4. close': '977.2000',
  '5. volume': '21616141'},
 '2022-04-19': {'1. open': '1005.0600',
  '2. high': '1034.9400',
  '3. low': '995.3250',
  '4. close': '1028.1500',
  '5. volume': '16615944'},
 '2022-04-18': {'1. open': '989.0300',
  '2. high': '1014.9200',
  '3. low': '973.4100',
  '4. close': '1004.2900',
  '5. volume': '17238407'},
 '2022-04-14': {'1. open': '999.2900',
  '2. high': '1012.7099',
  '3. low': '982.1900',
  '4. close': '985.0000',
  '5. volume': '19474135'},
 '2022-04-13': {'1. open': '981.0750',
  '2. high': '1026.2400',
  '3. low': '973.0970',
  '4. close': '1022.3700',
  '5. volume': '18373737'},
 '2022-04-12': {'1. open': '997.6400',
  '2. high': '1021.1921',
  '3. low': '976.6001',
  '4. close': '986.9500',
  '5. volume': '21992032'},
 '2022-04-11': {'1. open': '980.4000',
  '2. high': '1008.4681',
  '3. low': '974.6400',
  '4. close': '975.9300',
  '5. volume': '197857

### Let's go ahead and take a look at the size of our DataFrame

When it comes to calculating the daily rate of return, it's better to have a full year of data. The API docs 

In [371]:
2974/365

8.147945205479452

In [372]:
whole_df.shape

(2974, 2)

We have roughly 8 years worth of data for TSLA, but we only need one year. We will do 366 days instead of 365 so that we can find 365 results as the differences between the returns.

In [373]:
whole_df = whole_df[:366]

In [374]:
whole_df.head()

,date,close
0,2022-04-20,977.2000
1,2022-04-19,1028.1500
2,2022-04-18,1004.2900
3,2022-04-14,985.0000
4,2022-04-13,1022.3700


### Return One

Now we have the daily returns.

In [380]:
#Creating new column with prices from day before
whole_df['day_before'] = whole_df['close'].shift(-1)
#Changing the columns to floats
whole_df["close"] = pd.to_numeric(whole_df["close"], downcast="float")
whole_df["day_before"] = pd.to_numeric(whole_df["day_before"], downcast="float")
#Creating new column for the differences
whole_df['return'] = whole_df['day_before'] - whole_df['close']
#Inspecting
whole_df.head()

,date,close,return,day_before
0,2022-04-20,977.200012,50.950012,1028.150024
1,2022-04-19,1028.150024,-23.860046,1004.289978
2,2022-04-18,1004.289978,-19.289978,985.000000
3,2022-04-14,985.000000,37.369995,1022.369995
4,2022-04-13,1022.369995,-35.419983,986.950012


Inspecting the tail of the data: As expected, the last row does not have differences in data.

In [382]:
whole_df.tail()

,date,close,return,day_before
361,2020-11-11,417.130005,-6.770020,410.359985
362,2020-11-10,410.359985,10.900024,421.260010
363,2020-11-09,421.260010,8.690002,429.950012
364,2020-11-06,429.950012,8.139984,438.089996
365,2020-11-05,438.089996,NaN,NaN


Reording the columns of the dataframe. 

In [383]:
whole_df = whole_df[['date', 'close', 'day_before', 'return']]

In [384]:
whole_df.head()

,date,close,day_before,return
0,2022-04-20,977.200012,1028.150024,50.950012
1,2022-04-19,1028.150024,1004.289978,-23.860046
2,2022-04-18,1004.289978,985.000000,-19.289978
3,2022-04-14,985.000000,1022.369995,37.369995
4,2022-04-13,1022.369995,986.950012,-35.419983


In [470]:
whole_df['month_year'] = whole_df.date.str[:7]

In [471]:
whole_df.head()

,date,close,day_before,return,month_year
0,2022-04-20,977.200012,1028.150024,50.950012,2022-04
1,2022-04-19,1028.150024,1004.289978,-23.860046,2022-04
2,2022-04-18,1004.289978,985.000000,-19.289978,2022-04
3,2022-04-14,985.000000,1022.369995,37.369995,2022-04
4,2022-04-13,1022.369995,986.950012,-35.419983,2022-04


In [473]:
whole_df = whole_df.set_index(whole_df.month_year)
whole_df

,date,close,day_before,return,month_year
month_year,,,,,
2022-04,2022-04-20,977.200012,1028.150024,50.950012,2022-04
2022-04,2022-04-19,1028.150024,1004.289978,-23.860046,2022-04
2022-04,2022-04-18,1004.289978,985.000000,-19.289978,2022-04
2022-04,2022-04-14,985.000000,1022.369995,37.369995,2022-04
2022-04,2022-04-13,1022.369995,986.950012,-35.419983,2022-04
...,...,...,...,...,...
2020-11,2020-11-11,417.130005,410.359985,-6.770020,2020-11
2020-11,2020-11-10,410.359985,421.260010,10.900024,2020-11
2020-11,2020-11-09,421.260010,429.950012,8.690002,2020-11


### The interest rate on the 3 month Treasury Bill is going to be used as the Risk-Free Rate

This data can be pulled from the FRED API. Source: https://fred.stlouisfed.org/docs/api/fred/

In [476]:
url = 'https://api.stlouisfed.org/fred/series/observations?series_id=TB3MS&api_key=390054063dc48ea2c24e395c295ec168&file_type=json'
r = requests.get(url)
tb= r.json()
print(tb)

{'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'observation_start': '1600-01-01', 'observation_end': '9999-12-31', 'units': 'lin', 'output_type': 1, 'file_type': 'json', 'order_by': 'observation_date', 'sort_order': 'asc', 'count': 1059, 'offset': 0, 'limit': 100000, 'observations': [{'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-01-01', 'value': '0.72'}, {'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-02-01', 'value': '0.62'}, {'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-03-01', 'value': '0.24'}, {'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-04-01', 'value': '0.15'}, {'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-05-01', 'value': '0.16'}, {'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-06-01', 'value': '0.15'}, {'realtime_start': '2022-04-21', 'realtime_end': '2022-04-21', 'date': '1934-07-0

In [495]:
tb_list = tb['observations']

Now we have successfully created a DataFrame with all of the daily interest rates of the 3 month Treasury Bill Interest rate. We are only interested in the rates over the last year.

In [496]:
tb_df = pd.DataFrame(tb_list)
tb_df.head()

,realtime_start,realtime_end,date,value
0,2022-04-21,2022-04-21,1934-01-01,0.72
1,2022-04-21,2022-04-21,1934-02-01,0.62
2,2022-04-21,2022-04-21,1934-03-01,0.24
3,2022-04-21,2022-04-21,1934-04-01,0.15
4,2022-04-21,2022-04-21,1934-05-01,0.16


In [497]:
tb_df['date'] = tb_df.date.str[:7]

In [498]:
tb_df.head()

,realtime_start,realtime_end,date,value
0,2022-04-21,2022-04-21,1934-01,0.72
1,2022-04-21,2022-04-21,1934-02,0.62
2,2022-04-21,2022-04-21,1934-03,0.24
3,2022-04-21,2022-04-21,1934-04,0.15
4,2022-04-21,2022-04-21,1934-05,0.16


These columns need to be changed into datetime objects.

In [499]:
tb_df.realtime_start = pd.to_datetime(tb_df['realtime_start'])
tb_df.realtime_end = pd.to_datetime(tb_df['realtime_end'])


In [500]:
tb_df.head()

,realtime_start,realtime_end,date,value
0,2022-04-21,2022-04-21,1934-01,0.72
1,2022-04-21,2022-04-21,1934-02,0.62
2,2022-04-21,2022-04-21,1934-03,0.24
3,2022-04-21,2022-04-21,1934-04,0.15
4,2022-04-21,2022-04-21,1934-05,0.16


In [501]:
tb_df = tb_df[ tb_df['date'] >= '2021-04']

In [502]:
tb_df = tb_df.rename(columns = {'value':'Rf'})
tb_df.head()
             

,realtime_start,realtime_end,date,Rf
1047,2022-04-21,2022-04-21,2021-04,0.02
1048,2022-04-21,2022-04-21,2021-05,0.02
1049,2022-04-21,2022-04-21,2021-06,0.04
1050,2022-04-21,2022-04-21,2021-07,0.05
1051,2022-04-21,2022-04-21,2021-08,0.05


These are monthly dates per value.

In [503]:
tb_df.tail()

,realtime_start,realtime_end,date,Rf
1054,2022-04-21,2022-04-21,2021-11,0.05
1055,2022-04-21,2022-04-21,2021-12,0.06
1056,2022-04-21,2022-04-21,2022-01,0.15
1057,2022-04-21,2022-04-21,2022-02,0.33
1058,2022-04-21,2022-04-21,2022-03,0.44


In [504]:
tb_df.shape

(12, 4)

In [515]:
whole_df.shape

(366, 4)

In [508]:
tb_df = tb_df.set_index(tb_df.date)

In [509]:
tb_df.head()

,realtime_start,realtime_end,date,Rf
date,,,,
2021-04,2022-04-21,2022-04-21,2021-04,0.02
2021-05,2022-04-21,2022-04-21,2021-05,0.02
2021-06,2022-04-21,2022-04-21,2021-06,0.04
2021-07,2022-04-21,2022-04-21,2021-07,0.05
2021-08,2022-04-21,2022-04-21,2021-08,0.05


### Now the two Dataframes can be joined.

In [511]:
whole_df = whole_df.drop(columns='month_year')

In [530]:
new_df = whole_df.join(tb_df, on='month_year', how = 'right', lsuffix = 'L', rsuffix = 'R')

In [531]:
new_df.head()

,dateL,close,day_before,return,realtime_start,realtime_end,dateR,Rf
month_year,,,,,,,,
2022-03,2022-03-31,1077.599976,1093.989990,16.390015,2022-04-21,2022-04-21,2022-03,0.44
2022-03,2022-03-30,1093.989990,1099.569946,5.579956,2022-04-21,2022-04-21,2022-03,0.44
2022-03,2022-03-29,1099.569946,1091.839966,-7.729980,2022-04-21,2022-04-21,2022-03,0.44
2022-03,2022-03-28,1091.839966,1010.640015,-81.199951,2022-04-21,2022-04-21,2022-03,0.44
2022-03,2022-03-25,1010.640015,1013.919983,3.279968,2022-04-21,2022-04-21,2022-03,0.44


In [532]:
new_df = new_df.reset_index()

In [533]:
new_df.head()

,month_year,dateL,close,day_before,return,realtime_start,realtime_end,dateR,Rf
0,2022-03,2022-03-31,1077.599976,1093.989990,16.390015,2022-04-21,2022-04-21,2022-03,0.44
1,2022-03,2022-03-30,1093.989990,1099.569946,5.579956,2022-04-21,2022-04-21,2022-03,0.44
2,2022-03,2022-03-29,1099.569946,1091.839966,-7.729980,2022-04-21,2022-04-21,2022-03,0.44
3,2022-03,2022-03-28,1091.839966,1010.640015,-81.199951,2022-04-21,2022-04-21,2022-03,0.44
4,2022-03,2022-03-25,1010.640015,1013.919983,3.279968,2022-04-21,2022-04-21,2022-03,0.44


In [534]:
new_df = new_df[['dateL','return','Rf']]

In [538]:
new_df = new_df.rename(columns = {'dateL':'date'})

In [539]:
new_df['Rf_day_before'] = new_df['Rf'].shift(-1)

In [546]:
new_df.tail()

,date,return,Rf,Rf_day_before
248,2021-04-08,-12.830017,0.02,0.02
249,2021-04-07,20.650024,0.02,0.02
250,2021-04-06,-0.570007,0.02,0.02
251,2021-04-05,-29.299988,0.02,0.02
252,2021-04-01,6.179993,0.02,NaN


In [547]:
new_df.Rf = pd.to_numeric(new_df['Rf'])
new_df.Rf_day_before = pd.to_numeric(new_df['Rf_day_before'])

In [548]:
new_df['Rf_sub'] = new_df['Rf_day_before'] - new_df['Rf']

In [549]:
new_df.head()

,date,return,Rf,Rf_day_before,Rf_sub
0,2022-03-31,16.390015,0.44,0.44,0.0
1,2022-03-30,5.579956,0.44,0.44,0.0
2,2022-03-29,-7.729980,0.44,0.44,0.0
3,2022-03-28,-81.199951,0.44,0.44,0.0
4,2022-03-25,3.279968,0.44,0.44,0.0


In [550]:
new_df = new_df[['date', 'return', 'Rf_sub']]

In [552]:
new_df.tail()

,date,return,Rf_sub
248,2021-04-08,-12.830017,0.0
249,2021-04-07,20.650024,0.0
250,2021-04-06,-0.570007,0.0
251,2021-04-05,-29.299988,0.0
252,2021-04-01,6.179993,NaN


In [554]:
new_df.head()

,date,return,Rf_sub
0,2022-03-31,16.390015,0.0
1,2022-03-30,5.579956,0.0
2,2022-03-29,-7.729980,0.0
3,2022-03-28,-81.199951,0.0
4,2022-03-25,3.279968,0.0


There are some missing values in the dates because it covers a year, but only has 252 rows. The average daily return and average risk-free rate can now be calculated.

In [557]:
avg_daily_return = new_df['return'].mean()
avg_daily_return

-1.6192489862442017

In [559]:
risk_free_rate = new_df['Rf_sub'].mean()
risk_free_rate

-0.0016666666666666668

In [564]:
difference = avg_daily_return - risk_free_rate

TypeError: 'float' object is not iterable

### Objective One: Create a function where the input is number of shares, purchase price, and purchase date. The function should return the Sharpe Ratio. 

To get the sharpe calculation of one stock:

we have to take the mean price difference per day for the last year starting at the date of purchase

BUT we want to have a cumulative sharpe ratio for all of the stocks that we own so we want the function to take in an array that is 4 columns: ticker, purchase price, purchase date and number of shares. 

Then we want the function to take the last year of the prices and find the differences between each price day by day and then find the average of that. 

Then we want the function to save that number to the average_daily_return list. 
We want the function to do the same thing with the 3 month Treasury rate and create a list with one number per array row. 

Then we want to take the sum of the average_daily_return list.
Then we want to take the sum of the 3mo_tbill list.

Then we want to take the avg_daily_return_sum minus the 3mo_Tbill_sum to find the numerator.
Then we want to take the standard deviation of the numerator to find the denominator. 
Then we want to divide the numerator by the denominator to find the sharpe ratio of the portfolio.